In [1]:
# load module
import xarray as xr
import numpy as np

In [2]:
# compute closest ocean coastal cell
def get_dist_ocean_point_1d(xlon, ylat, lon1d, lat1d):
    '''Haversine formula'''
    earth_radius_miles = 6371.0
    dlat = lat1d[:] - ylat
    dlon = lon1d[:] - xlon
    a = np.square(np.sin(dlat/2.0)) + np.cos(ylat) * np.cos(lat1d[:]) * np.square(np.sin(dlon/2.0))
    great_circle_distance = 2 * np.arcsin(np.sqrt(a))
    d = earth_radius_miles * great_circle_distance
    return d

In [ ]:
def get_coastal_msk(mask_in,lewp):
    nj_out=mask_in.shape[0]
    ni_out=mask_in.shape[1]

    if lewp:
        mask=np.zeros(shape=(nj_out,ni_out+2))
        mask[:,1:-1]=mask_in
        mask[:, 0]=mask[:,-2]
        mask[:,-1]=mask[:, 1]
        xslc=slice(0,ni_out)
    else:
        mask=mask_in
        xslc=slice(1,-1)
        
    mask_coast=np.zeros(shape=(nj_out,ni_out))
    mask_coast[1:-1,xslc]= ( mask[1:-1,1:-1] + 
                             mask[0:-2,1:-1] + mask[2::,1:-1] + mask[1:-1,0:-2] + mask[1:-1,2::] +
                             mask[0:-2,0:-2] + mask[2::,2::]  + mask[2::,0:-2]  + mask[0:-2,2::] ) * mask[1:-1,1:-1]
    mask_coast[(mask_coast > 1) & (mask_coast < 9)] = 10
    mask_coast[mask_coast!=10]=np.nan
    mask_coast=mask_coast.astype(np.float32)
    mask_coast[mask_coast==10]=1
    
    return mask_coast

In [ ]:
def get_1d_data(ds,ptslst):
    print('get lon 1d')
    lon1d=np.radians(ds.lon.values[ptslst[0],ptslst[1]])
    print('get lat 1d')
    lat1d=np.radians(ds.lat.values[ptslst[0],ptslst[1]])
    print('get thickness 1d')
    h1d=ds.thickness.values[ptslst[0],ptslst[1]]
    print('get mask ice')
    msk1d=ds.mask.values[ptslst[0],ptslst[1]]
    return lon1d,lat1d,h1d,msk1d

In [3]:
cfin ='BedMachineAntarctica-v3_ll.nc'
cfout='eANT025.L121_icbh_b0.6.1_v0.0.nc'

# ## Input data
print('open input file')
ds_in=xr.open_dataset(cfin)

open input file


In [75]:
print('get coastal trg mask')
mask_in=ds_in.msk_ice.values[:,:]
mask_coast_in=get_coastal_msk(mask_in,False)

get coastal trg mask


In [73]:
print('get BedMachine coastal input points')
ij_lst=np.where(mask_coast_in == 1)
print(len(ij_lst[0]))

get BedMachine coastal input points
472082


In [29]:
lon_in_1d,lat_in_1d,h_in_1d,msk_in_1d=get_1d_data(ds_in,ij_lst)

get lon 1d
get lat 1d
get thickness 1d
get mask ice


In [76]:
# ## Output data
cftrg='eANT025.L121_mesh_mask_b0.6.1.nc'

print('open output file')
# define coastal mask
ds_trg=xr.open_dataset(cftrg)
e1t_trg=ds_trg.e1t.values.squeeze()

lon_trg=ds_trg.nav_lon.values.squeeze()
lat_trg=ds_trg.nav_lat.values.squeeze()
lon_trg_rad=np.radians(lon_trg)
lat_trg_rad=np.radians(lat_trg)

print('ni/nj out')
nj_out=lon_trg.shape[0]
ni_out=lon_trg.shape[1]

mask_trg=np.zeros(shape=(nj_out,ni_out+2))
mask_coast_trg=np.zeros(shape=(nj_out,ni_out))

print('get coastal mask')
mask_trg=ds_trg.tmask.values[0,0,:,:].squeeze()
mask_coast_trg=get_coastal_msk(mask_trg,True)

print('get 1d array of coastal points')
iy_trg,ix_trg=np.where(mask_coast_trg==1)
mask_coast_trg_1d=mask_coast_trg[iy_trg,ix_trg]
lon_trg_rad_1d=lon_trg_rad[iy_trg,ix_trg]
lat_trg_rad_1d=lat_trg_rad[iy_trg,ix_trg]

open output file
ni/nj out
get coastal mask
get 1d array of coastal points


In [77]:
hicb_trg=np.zeros(shape=(nj_out,ni_out))
mskicb_trg=np.zeros(shape=(nj_out,ni_out))
ndaticb_trg=np.zeros(shape=(nj_out,ni_out))
ij_miss=np.where(mask_coast_trg == 1)
print(len(ij_miss[0]))
for ipt,pts in enumerate(ij_miss[0]):
    if ipt%100 == 0:
        print(ipt)
    iiy=ij_miss[0][ipt]
    iix=ij_miss[1][ipt]
    lon_trg_rad_0d=lon_trg_rad[iiy,iix]
    lat_trg_rad_0d=lat_trg_rad[iiy,iix]
    dist=get_dist_ocean_point_1d(lon_trg_rad_0d, lat_trg_rad_0d, lon_in_1d, lat_in_1d)
    if np.min(dist) < 100 :
        search_radius=e1t_trg[iiy,iix]/2/1000 # in km
        zh_in=h_in_1d[dist<=np.min(dist)+search_radius]
        zmsk_in=msk_in_1d[dist<=np.min(dist)+search_radius]
        hicb_trg[iiy,iix]=np.mean(zh_in[zmsk_in==3])
        if hicb_trg[iiy,iix]== 0:
            print(iiy,iix,h_in_1d[dist<=np.min(dist)+search_radius])
        if np.isnan(hicb_trg[iiy,iix]):
            hicb_trg[iiy,iix]=50.0
            mskicb_trg[iiy,iix]=2
        else:
            mskicb_trg[iiy,iix]=1
            ndaticb_trg[iiy,iix]=len(zh_in[zmsk_in==3])
    

3912
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900


In [78]:
print('write output')
# ## Write output
# define data with variable attributes
data_vars = {'icbh':(['y','x'], hicb_trg,
                          {'units': 'm', 
                           'long_name':'iceberg maximal thickness'}),
             'coast':(['y','x'], mask_coast_trg,
                         {'units': '[]', 
                           'long_name':'coastl point mask'}),
             'nisfpts':(['y','x'], ndaticb_trg,
                         {'units': '[]', 
                           'long_name':'number of isf point in determining icb thickness'}),
             'icbmsk':(['y','x'], mskicb_trg,
                          {'units': '[]', 
                           'long_name':'iceberg type mask '
                           '(0:no calving point, '
                           ' 1:calving from at least one floating ice shelf,'
                           ' 2:from grounded ice sheet)'}),
                          }

# define coordinates
coords = {'nav_lon': (['y','x'], lon_trg),
          'nav_lat': (['y','x'], lat_trg)}

# define global attributes
attrs = {'source':cfin, 
         'method':('Average of all BedMachine coastal points in a circle of d km center on each coastal NEMO T points.'
         'd = 5 + min(gread_circle_distance(NEMO T point, List of BedMachine coastal points))')
         }

# create dataset
ds_out = xr.Dataset(data_vars=data_vars, 
                coords=coords, 
                attrs=attrs)

ds_out.to_netcdf(cfout)

write output


In [ ]:
# debug
#ds_calv=xr.open_dataset('calving_eANT025.L121.nc')
#ij_clv=np.where(ds_calv.soicbclv[0,:,:]!= 0)
#for ipt, pts in enumerate(ij_clv[0]):
#    iiy=ij_clv[0][ipt]
#    iix=ij_clv[1][ipt]
#    if hicb_trg[iiy,iix]==0 :
#        print(iiy,iix,ds_calv.soicbclv.values[:,iiy,iix],hicb_trg[iiy,iix],mask_coast_trg[iiy,iix])
#ds_bm=xr.open_dataset('BedMachineAntarctica-v3.nc')
#hdbg=ds_bm.thickness.values[ds_bm.mask.values > 1]
#np.min(hdbg)